# EDA of the spotify dataset from kaggle

Columns description:
- track_id: The Spotify ID for the track
- artists: The artists' names who performed the track. If there is more than one artist, they are separated by a ;
- album_name: The album name in which the track appears
- track_name: Name of the track
- popularity: The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity.
- duration_ms: The track length in milliseconds
- explicit: Whether or not the track has explicit lyrics (true = yes it does; false = no it does not OR unknown)
- danceability: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable
- energy: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale
- key: The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1
- loudness: The overall loudness of a track in decibels (dB)
- mode: Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0
- speechiness: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks
- acousticness: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic
- instrumentalness: Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content
- liveness: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live
- valence: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry)
- tempo: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration
- time_signature: An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of 3/4, to 7/4.
- track_genre: The genre in which the track belongs


### Preparation 
Goal: Create a suitable model that predicts the popularity of a song.  
Possible type of prediction model: *Regression* (number specific, precise, possibly unstable) or *Classification* (popular/not popular, binary, less precise, possibly more stable).  
Possible use case: A marketing firm that wants to know how much money to spend on upcoming songs depending on predicted popularity to better plan the expenses and profits.

### Metrics/Evaluation
For regression models: Mean absolute error (MAE), R² score  
For classification models: Precision score, recall score, f1 score


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# uncomment for data retrieval:
#%run .\src\data_retrieval\download_from_kagglehub.py

In [ ]:
df = pd.read_csv('data/spotify_dataset.csv', index_col=0)

# show 10 random (sample) rows
df.sample(10)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
print(f'{"columns":<20}{"# unique values"}')
print('-'*40)
for column in df.columns:
    print(f'{column:<20}{df[column].nunique()}')

In [ ]:
df.dtypes

In [ ]:
# crate COLS lists für categorical and numerical columns
# remove columns that are not needed for the analysis

# removed track_id from CAT_COLS
CAT_COLS = [
    'artists',
    'album_name',
    'track_name',
    'explicit',
    'track_genre'
]

NUM_COLS = [
    'popularity',
    'duration_ms',
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'time_signature'
]

In [ ]:
# check exact nan values in columns: artists, album_name, track_name

for col in ['artists', 'album_name', 'track_name']:
    display(df[df[col].isna()])

In [ ]:
# Only 1 row of the dataset is affected by nan values, drop it

print(df.shape)
df = df.dropna()
print(df.shape)

In [ ]:
# Check for duplicate rows
print("Duplicate rows count:", df.duplicated().sum())

# Check for duplicate track names
print("Duplicate track names:", df.duplicated(subset=['track_name']).sum())

In [ ]:
# Look at duplicate track names
duplicate_names = df[df.duplicated(subset=['track_name'], keep=False)]

# Group by track name and show counts
print(duplicate_names['track_name'].value_counts().head(20))

In [ ]:
# check individual duplicates
df[df['track_name'] == 'Run Rudolph Run']

It seems that the duplicates are mostly identical across all columns, only some of them seem to be different. These are the ones I will keep, the others will be dropped.

In [ ]:
# Drop duplicates where all relevant columns are the same
relevant_cols = ['artists', 'track_name', 'duration_ms', 'explicit',
                'danceability', 'energy', 'key', 'loudness', 'mode',
                'speechiness', 'acousticness', 'instrumentalness',
                'liveness', 'valence', 'tempo', 'time_signature'] 

df_clean = df.copy()
df_clean = df_clean.drop_duplicates(subset=relevant_cols)

# check individual duplicates again
df_clean[df_clean['track_name'] == 'Run Rudolph Run']

In [ ]:
# check shape after dropping duplicates
print(df_clean.shape)
print(df.shape[0] - df_clean.shape[0], "duplicates dropped")


In [ ]:
# Check num columns
df_clean[NUM_COLS]

### Notes about NUM_COLS
The numerical columns have some special ones:
- "duration_ms" is used as an integer, but actually represents milliseconds (time type) -> may be useful to change to something like minutes (more common for song lengths) --> actually **categorical**, no inherent ranking
- "key" (Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1) --> actually **categorical**, no inherent ranking
- "mode" (major=1 or minor=0), binary categorical
- "tempo" actually means the BPM (Beats per minute, with float number detail) and is rather some kind of categorical ranking number (the higher the number the faster a song)
- "time signature" (An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of 3/4, to 7/4) --> actually **categorical** number, no inherent ranking

- values of 0 in "tempo" and "time signature" probably refer to **"unknown"** since they cannot have values of 0 from their description


In [ ]:
# Check cat columns
df_clean[CAT_COLS]

### Notes about CAT_COLS
The categorical columns have a special one:
- "explicit" is of type boolean (False/True) and could be transformed to binary (0, 1), to use as numerical.
Rest of the cat columns look right.


In [ ]:
# check for outliers via the IQR method

num_cols_check = ['popularity',
                  'danceability',
                  'energy',
                  'loudness',
                  'speechiness',
                  'acousticness',
                  'instrumentalness',
                  'liveness',
                  'valence',]
# the other num cols are categorical and should not be checked for outliers

outlier_summary = []
for col in num_cols_check:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 10 * IQR
    upper = Q3 + 10 * IQR

    mask = (df_clean[col] < lower) | (df_clean[col] > upper)
    n_outliers = mask.sum()

    outlier_summary.append({
        'column': col,
        'total': len(df_clean),
        'outliers': n_outliers
    })

outlier_df = pd.DataFrame(outlier_summary)
print(outlier_df)

In [ ]:
fig, ax = plt.subplots(ncols=3, nrows=3, figsize=(12, 10))

# flatten = turns the subplots 3x3 array into a 1D array (list-like) to help with iteration inside the following loop
ax = ax.flatten()

for i, col in enumerate(num_cols_check):
    ax[i].boxplot(df_clean[col])
    ax[i].set_title(f'Boxplot of {col}')
    ax[i].set_ylabel(col)

plt.tight_layout()

### Notes about outliers:
- Tried ranging numbers to check for outliers for the IQR method from low as 1.5 to 10 (as in the code above currently)
- With higher IQR ranges the only two columns showing possible "outliers" (or natural variance) are "speechiness" and "instrumentalness"
- But (and the boxplot also support it visually) most columns have a wide range of "outliers" and it seems like they are rather natural variance data to be kept  

Conclusion: There seem to be no clear outliers here, will keep all data for further analysis and modeling.

In [ ]:
# check correlation between numerical columns
corr = df_clean[NUM_COLS].corr()

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', center=0, square=True, cbar_kws={'shrink': .8}, ax=ax)

### Notes about stronger correlations:
- acousticness and loudness (negative correlation, -0.73)
- loudness and energy (positive correlation, 0.76)

"Acoustical" songs are probably recorded live or have less sound enhancement (compression, other sound engineered techniques to make a song louder).
Louder songs can probably lead to a stronger reception of "energy" (energizing somebody) like it is felt with sound enhanced club songs / electronic songs that are often mastered to be as loud as possibly (db wise).

For later machine learning model: 
- Some models safely get around intercorrelations (tree-based models)
- For linear models, a ridge or lasso regulation may be suitable
- PCA is also an option

In [ ]:
# check individual scatterplots of the most correlated columns
fix, ax = plt.subplots(ncols=2, figsize=(10, 5))
sns.scatterplot(x='loudness', y='acousticness', data=df_clean, alpha=0.5, ax=ax[0]);
sns.scatterplot(x='loudness', y='energy', data=df_clean, alpha=0.5, ax=ax[1]);

plt.tight_layout()


In [ ]:
# check individual histplots for better feel of numerical columns
fig, ax = plt.subplots(ncols=2, nrows=7, figsize=(12, 10))
ax = ax.flatten()

for i, col in enumerate(NUM_COLS):
    sns.histplot(df_clean[col], bins=50, kde=True, color='blue', alpha=0.7, ax=ax[i])
    ax[i].set_ylabel(col)

plt.tight_layout();

### Some notes about histplots:
- popularity: Most are actually 0 or around it (these songs have maybe not been played for a while?)
- instrumentalness: most are also 0 or around it (maybe was not available to get most of the time?)
- acousticness: most are 0, rest is quite equally distributed
- valence: wide range of almost equal distribution across all data
- time_signature: most songs are in the 4/4 time_signature (which makes sense, its probably the most common time signature in music)

In [ ]:
## further check of popularity as possible target 
# check distribution of popularity values
popularity_counts = pd.crosstab(index=df_clean['popularity'], columns='count').sort_values(by='count', ascending=False)
popularity_counts


### Notes about popularity as possible target
- Many datapoints contain popularity values of 0 which is quite unbalanced and the model might be biased towards predicting lower values
- Popularity scales from 0 to 100 and might be hard to predict precisely with a regression model
- Also values of 0 can also mean the track has not been played for a while, or it was recently added to Spotify an hence has not been played at the time of the data retrieval
- The popularity algorithm focues on **how often a song has recently been played** so new songs that are being played are heavily favored

For later machine-learning model:  
- Resampling techniques could help to balance the the values, or using a balanced weight parameter in specific models
- Might be better to recategorize popularity and use a classification model in the first place

In [ ]:
# check popularity vs. categorical columns with low unique value counts
for col in ['explicit', 'key', 'mode', 'time_signature']:   
    display(pd.crosstab(df['popularity'], df_clean[col], normalize='index').sort_values(by='popularity'))

In [ ]:
# check scatterplots of popularity vs. numerical features
fig, ax = plt.subplots(ncols=2, nrows=7, figsize=(12, 20))
ax = ax.flatten()

for i, col in enumerate(NUM_COLS):
    ax[i].scatter(df_clean[col], df_clean['popularity'], alpha=0.3)
    ax[i].set_title(f'Popularity vs. {col}')
    ax[i].set_xlabel(col)
    ax[i].set_ylabel('popularity')
    ax[i].set_ylim(0, 120)

plt.tight_layout()

### Notes about scatterplots of popularity vs. numerical columns:  
- mostly wide ranges of popularity values across most of the numerical columns
- the "actual categorical" columns like mode and key also have a wide variance
- the speechiness column has an interesting distribution: A higher speechiness shows less popularity on the higher end and a more focused distribution on the middle to lower end. This probably means that songs that are highly vocal (acapella, etc.) with less or none instrumental aspects are less popular. This makes sense, to have a good balance of instrumentals and vocals seems logical. It shows a stronger emphasis on instrumentalness than I thought.

In [ ]:
# check correlation of all columns with popularity specifically
# Convert to DataFrame for heatmap
corr_pop = df_clean.corr(numeric_only=True)['popularity'].sort_values(ascending=False)
print(f'Correlation of numeric columns with popularity in:\n{corr_pop}')


No very strong correlations with popularity, highest values are 0.09 for loudness and -0.175 for instrumentalness.  
Categorical columns could also lead a stronger role. Will see in later machine-learning evaluation.

### Categorize target column "popularity":
Decided to go for a classification model. Popularity as target column needs to be **categorized** for that, therefore:
- "New" = values of -1 or 0, represents new songs or songs that have not been played on spotify yet for some reason
- "Low", "Medium", "High" = by cutting 1/3rd (at 33, 66 and 100) the value ranges represent low, medium and high popularity


In [ ]:
# Use pd.cut for equal-width bins to categorize popularity

df_clean['popularity_cat'] = pd.cut(df_clean['popularity'],
                                    bins=[-1, 0, 33, 66, 100],
                                    labels=['New', 'Low', 'Medium', 'High'])

print(df_clean['popularity_cat'].value_counts())
print(df_clean.columns)

The categorization is unbalanced considering medium and low are dominating the popularity values (which has already been seen in the eda), but
a balance class weight in a classification model probably fixes that. The categorization shouldn't become too complex. Will work with that for now.

In [ ]:
# safe cleaned dataset as csv to use in machine-learning process
df_clean.to_csv('./data/spotify_cleaned.csv')